# Introduction to Neural Networks

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

![dense](../img/dogcat.gif)

## Agenda

SWBAT:

- describe the basic structure of densely connected neural networks;
- describe the various activation functions that are used in neural networks;
- describe the concept of backpropagation;
- explain the use of gradient descent in neural networks.

## Background

Neural networks have been around for a while. They are over 70 years old, dating back to  their proposal in 1944 by Warren McCullough and Walter Pitts. These first proposed neural nets had thresholds and weights, but no layers and no specific training mechanisms.

The "perceptron", the first trainable neural network, was created by Frank Rosenblatt in 1957. It consisted of a single layer with adjustable weights in the middle of input and output layers.

![peceptron](../img/nn-diagram.png)

## Relation to Previous Models

### Logistic Regression

Recall logistic regression:

Think of the weights as the betas and the activation function as the sigmoid function!

### Stacking Ensembles

Recall model stacking:

Various base models' predictions are fed into a "meta-estimator" that is trained to aggregate them optimally. This is analogous to the multiple **layers** of a neural network.

## Basic Architecture

For our DS purposes, we'll generally imagine our network to consist of only a few layers, including an input layer (where we feed in our data) an output layer (comprising our predictions). Significantly, there will also (generally) be one or more layers of neurons between input and output, called **hidden layers**.

One reason these are named hidden layers is that what their output actually represents in not really known.  The activation of node 1 of the first hidden layer may represent a sequence of pixel intensity corresponding to a horizontal line, or a group of dark pixels in the middle of a number's loop. 

![dense](../img/Deeper_network.jpg)

Because we are unaware of how exactly these hidden layers are operating, neural networks are considered **black box** algorithms.  You will not be able to gain much inferential insight from a neural net.

Each of our pixels from our digit representation goes to each of our nodes, and each node has a set of weights and a bias term associated with it.

## Inspiration from Actual Neurons

The composition of neural networks can be **loosely** compared to a neuron.
![neuron](../img/neuron.png)

Neural networks draw their inspiration from the biology of our own brains, which are of course also accurately described as 'neural networks'. A human brain contains around $10^{11}$ neurons, connected very **densely**.

This is a loose analogy, but can be a helpful **mnemonic**. The inputs to our node are like inputs to our neurons. They are either direct sensory information (our features) or input from other axons (nodes passing information to other nodes). The body of our neuron (soma) is where the signals of the dendrites are summed together, which is loosely analogous to our **collector function**. If the summed signal is large enough (our **activation function**), they trigger an action potential which travels down the axon to be passed as output to other dendrites. See [here](https://en.wikipedia.org/wiki/Neuron) for more. 

## Forward Propagation

Let's first look at **forward propagation** on the level of the perceptron.

We will use the built-in dataset of handwritten numbers from `sklearn`, which comes from the UCI Machine Learning collection [digits source](https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits). Each record is a 64-bit (8x8) image of a handwritten number between 0 and 9. Each pixel value (a number between 0 and 16) represents the relative brightness of the pixel. 

In [ ]:
digits = load_digits()
flat_image = np.array(digits.data[0]).reshape(digits.data[0].shape[0], -1)
eight_by_eight_image = digits.images[0]

It is similar to the famous [**MNIST**](http://yann.lecun.com/exdb/mnist/index.html) dataset which is sometimes referred to the ["hello world" of computer vision](https://www.kaggle.com/c/digit-recognizer).  

Let's look at one digit:

In [ ]:
digits = load_digits()
eight_by_eight_image = digits.images[0]

In [ ]:
imgplot = plt.imshow(eight_by_eight_image, cmap='Greys')

In [ ]:
# look at the matrix below and make sure you see how the large numbers 
# correspond to darker shades in the image above

eight_by_eight_image

When passing the data into our perceptron, we will **flatten** the image into a 64x1 array.

In [ ]:
digits.data[0]

In [ ]:
digits.data[0].shape[0]

In [ ]:
flat_image = np.array(digits.data[0]).reshape(digits.data[0].shape[0], -1)

In [ ]:
flat_image.shape

### Weights

Our weights vector will have the same number of weights as pixels.

![weights](../img/log-reg-nn-ex-w.png)

We will instantiate our weight with small random numbers.


In [ ]:
w = np.random.uniform(-.1, .1, (flat_image.shape[0], 1))
w[:5]

We'll set our bias term to 0:

In [ ]:
b = 0

### Summation

![sum](../img/log-reg-nn-ex-sum.png)

Our inputs, the pixels, each are multiplied by their respective weights and then summed together with the bias. 

This amounts to the dotproduct of the pixel value and the weights.

In [ ]:
z = flat_image.T.dot(w) + b
z

### Activation

![activation](../img/log-reg-nn-ex-a.png)

Then we pass it into an activation function. The activation function converts our summed inputs into an output, which is then passed on to other nodes in hidden layers, or as an end product in the output layer. This can loosely be thought of as the action potential traveling down the axon.

When we build our models in `keras`, we will specify the activation function of both hidden layers and output.

We have a suite of activation functions to choose from.

#### Sigmoid

$f(x) = \frac{1}{1+e^{-z}}$

In [ ]:
# Z is the input from our collector, the sum of the weights
# multiplied by the features and the bias

def sigmoid(z):
    '''
    Input the sum of our weights times the pixel intensities, plus the bias
    Output a number between 0 and 1.
    
    '''
    return 1/(1+np.e**(-z))

In [ ]:
X = np.linspace(-10, 10, 20000)
sig = sigmoid(X)

plt.plot(X, sig);

#### tanh

$f(x) = tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$

The hyperbolic tangent function is a shifted version of the sigmoid. The inflection point passes through 0,0 instead of 0,.5, and the output is between -1 and 1.  This means the mean of the output is centered around 0, which can make learning in the next layer easier. tanh is almost always better in a **hidden layer** than sigmoid because it [speeds up learning](https://stats.stackexchange.com/questions/330559/why-is-tanh-almost-always-better-than-sigmoid-as-an-activation-function). For the output layer, however, sigmoid makes sense for binary outcomes.

In [ ]:
def tanh(z):
    return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))

In [ ]:
# Coding tanh:

X = np.linspace(-10, 10, 20000)
y_tanh = tanh(X)

plt.plot(X, y_tanh);

One problem with tanh (and sigmoid) is that if our input is large, the slope of the activation function flattens out.  When conducting backpropagation, we will use the derivative of the activation function as one of our terms multiplied by a learning rate to determine how big a step to take when adjusting our weights. If our derivative is close to zero, the step will be very small, so the speed of our learning will be very slow.  This is called the **vanishing gradient** problem.

#### ReLU

$f(x) = 0$ if $x\leq 0$; $f(x) = x$ otherwise

ReLU is a commonly used and effective activation function because of speed.  Given that the **output** is zero when negative, some nodes become inactive (i.e. produce an output of 0).  Zero outputs take little computational power. Also, the constant gradient leads to faster learning in comparison to sigmoid and tanh, which come close to 0 with large positive and large negative values. Since the speed of our network is linked to the derivative, a derivative close to zero will result in slow learning.

See also [this page on stackexchange](https://stats.stackexchange.com/questions/126238/what-are-the-advantages-of-relu-over-sigmoid-function-in-deep-neural-networks).

In [ ]:
def relu(z):
    if z <= 0:
        return 0
    else:
        return z

In [ ]:
# Coding ReLU:

X = np.linspace(-10, 10, 200)

y_relu = [relu(x) for x in X]

plt.plot(X, y_relu);

#### Softmax

$\large f(x_0) = \frac{e^{x_0}}{\Sigma_{x\epsilon X}e^x}$

Because this function relates each value to the sum of all values, this is the appropriate activation in the output layer for **multi-class** classification problems. We can interpret the function as outputting the probabilities of belonging to each class.

There are other activation functions; [see here](https://towardsdatascience.com/comparison-of-activation-functions-for-deep-neural-networks-706ac4284c8a). 

Our nodes will be taking in input from multiple sources. Let's add the entire training set as our input. 


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(digits.data,
                                                    digits.target,
                                                    random_state=42,
                                                    test_size=0.2)
X_train.shape

In [ ]:
X_train[0, :].reshape(8, 8)

In [ ]:
z_0 = X_train.dot(w)+b
z_0.shape

In [ ]:
a_0 = sigmoid(z_0)
a_0

In [ ]:
a_0_relu = [relu(z) for z in z_0]
a_0_relu[:10]

In [ ]:
w_1 = np.random.normal(0, 0.1, (X_train.shape[1], 4))
w_1.shape

In [ ]:
b_1 = 0

In [ ]:
z_1 = X_train.dot(w_1) + b_1
z_1

In [ ]:
a_1 = sigmoid(z_1)
a_1

Now each of these neurons has a set of weights and a bias associated with it.

In [ ]:
w_1 = np.random.normal(0, 0.1, (a_1.shape[1], 1))

w_1.shape

In [ ]:
b_1 = 0

In [ ]:
z_1 = a_1.dot(w_1)

In [ ]:
output = sigmoid(z_1)
y_pred = output > .5
y_hat = y_pred.astype(int)
y_hat[:5]

## Back propagation

After a certain number of data points have been passed through the model, the weights will be *updated* with an eye toward optimizing our loss function. (Thinking back to biological neurons, this is like revising their activation potentials.) Typically, this is  done  by using some version of gradient descent.

![bprop](../img/BackProp_web.png)

### Loss Function

[Neural networks are much like computational graphs](https://medium.com/tebs-lab/deep-neural-networks-as-computational-graphs-867fcaa56c9).

And computational graphs can be used [to approximate *any* function](http://neuralnetworksanddeeplearning.com/chap4.html).

If we're thinking of networks, then, as function approximators, of course we'll want to know how good the approximation is. And so once again we have the idea of a [loss function](https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html), which is of course what licenses our thinking of networks as models in the first place.

Many loss functions are available. Your choice will depend in part on whether you're doing a regression or classification problem.

Regression:

- mean / median absolute error
- mean / median squared error
- [Huber loss](https://en.wikipedia.org/wiki/Huber_loss)

Classification:

- Crossentropy
- [Hinge loss](https://en.wikipedia.org/wiki/Hinge_loss)
- [Kullback-Leibler divergence](https://www.countbayesie.com/blog/2017/5/9/kullback-leibler-divergence-explained)

The loss function tells us how well our model performed by comparing the predictions to the actual values.

When we train our models with `keras`, we will watch the loss function's progress across epochs.  A decreasing loss function will show us that our model is **improving**.

The loss function is associated with the nature of our output. In logistic regression, our output was binary, so our loss function was the negative loglikelihood, aka **cross-entropy**.

$$ \Large -\ loglikelihood = -\frac{1}{m} * \sum\limits_{i=1}^m y_i\log{p_i} + (1-y_i)\log(1-p_i) $$
    

In [ ]:
y_train

In [ ]:
y_train = y_train %2

In [ ]:
y_train

In [ ]:
y_train*np.log(output) + (1-y_train) * np.log(1-output)

In [ ]:
neg_ll = -1/len(y_train)*np.sum(y_train*np.log(output) + (1-y_train) * np.log(1-output))
neg_ll

For continuous variables, the loss function we have relied on is [MSE or MAE](http://rishy.github.io/ml/2015/07/28/l1-vs-l2-loss/).

Good [resource](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/) on backpropogation with RMSE loss function.

Here is a good summary of different [loss functions]( https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html):
   

## Gradient Descent, Epochs, and Batches

We not only use the the loss function to see how our model is improving; we also use it to update our parameters. The gradient of the loss function is calculated in relation to each parameter of our neural net.

For a deep dive into the fitting process, reference Chapter 11 in [Elements of Statistical Learning](https://web.stanford.edu/~hastie/ElemStatLearn/printings/ESLII_print12.pdf)

Gradient descent can be performed in several different ways.  Unlike the `sklearn` implementation of linear regression, which finds the minimum of the loss with a closed form solution, neural networks move down the gradient **incrementally**.

When we fit our neural nets in Keras, we can set the hyperparameter `verbose` equal to 1, and we will see progress through **epochs**. Setting `verbose` to 2 will show just the epoch numbers as they progress.

At the end of each epoch, **all examples** from are training set have passed through the network.

Different types of gradient descent update the parameters at different times.

### Batch Gradient Descent

The gradient is calculated across all values.  We can find the direction of the gradient, and proceed directly towards the minimum.

The weights are updated with regard to the cost at the **end of an epoch** after all training elements have passed through.

### Stochastic Gradient Descent

Updating the weights after all training examples have passed through can be detrimentally slow.  

SGD updates the weights after each training **example**. SGD requires fewer epochs to achieve quality coefficients. This speeds up gradient descent [significantly](https://machinelearningmastery.com/gradient-descent-for-machine-learning/).

### Mini-Batch Gradient Descent

In mini-batch, we pass a batch, calculate the gradient, update the params, then proceed to the next batch. It combines the advantages of batch and stochastic gradient descent: it is faster than SGD since the updates are not made with each point, and more computationally efficient than batch, since we don't have to hold all training examples in memory.

[Good comparison of types of Gradient Descent and batch size](https://machinelearningmastery.com/gentle-introduction-mini-batch-gradient-descent-configure-batch-size/)

## Optimizers

One of the levers we can tweak are the optimizers which control how the weights and biases are updated.

For stochastic gradient descent, the weights are updated with a **constant** learning rate (alpha) after every record.  If we specify a batch size, the constant learning rate is multiplied by the gradient across the batch. 

Other optimizers, such as **Adam** ("Adaptive Moment Estimation"), update the weights in different ways. For Adam,
> A learning rate is maintained for each network weight (parameter) and separately adapted as learning unfolds. See [here](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/).

![backprop](../img/ff-bb.gif)

The graphic above can be a bit frustrating since it moves fast, but follow the progress as so:

Forward propagation with the **blue** tinted arrows computes the output of each layer: i.e. a summation and activation.

Backprop calculates the partial derivative (**green** circles) for each weight (**brown** line) and bias.

Then the optimizer multiplies a **learning rate** ($\eta$) to each partial derivative to calculate a new weight which will be applied to the next batch that passes through.